# LOCALIZACIONES DEFINIDAS
posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
importar librerias

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

# FUNCIONES DE LLAMADA A LA API

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = LaVariableQueQuierasSinComillas)

In [11]:
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [12]:
def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    adress = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["formattedAddress"][0])      
    response2 = [name , type_, cc, adress,  lat, lng ]

    return response2

In [13]:
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country", 3:"Adress", 4:"Latitude", 5:"Longitude", })
    return df1

In [14]:
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [15]:
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [16]:
def df_final1(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country", "Adress","Latitude","Longitude","location"]]
    return df_terminado

# FUNCTIONS TO GET CONDITIONS OF A LOCATION

# SCHOOLS

In [17]:
Schools = ["Nursery School","Elementary School","Middle School","High School"]
def get_schools(location, Schools):
    Nursery_sch = df_final1(url, location, Schools[0])
    Elementary_sch = df_final1(url, location, Schools[1])
    Middle_sch = df_final1(url, location, Schools[2])
    High_sch = df_final1(url, location, Schools[3])
    all_schools_bad_index = pd.concat([Nursery_sch, Elementary_sch, Middle_sch, High_sch])
    all_schools = all_schools_bad_index.reset_index(drop=True)
    return all_schools

# DESIGNING COMPANIES

In [18]:
def get_design_studio(location):
    DesignCo = df_final1(url, location, "Design Studio")
    return DesignCo

# TECH STARTUPS

In [19]:
def get_tech_startups(location):
    TechStaUp = df_final1(url, location, "Tech Startup")
    return TechStaUp
    

# LEISURE

In [20]:
leisure = ["Nightclub", "Piano Bar", "Pub", "Irish Pub"]
def get_leisure(location, leisure):
    Disco = df_final1(url, location, leisure[0])
    Piano_bar = df_final1(url, location, leisure[1])
    Pub = df_final1(url, location, leisure[2])
    irish_pub = df_final1(url, location, leisure[3])
    all_leisure_bad_index = pd.concat([Disco, Piano_bar, Pub, irish_pub])
    leisure = all_leisure_bad_index.reset_index(drop=True)
    return leisure

# TRAVELLING STATIONS

In [21]:
stations = ["Metro Station", "Light Rail Station", "Bus Station"  ]
def get_closeby_stations(location, stations):
    Metro = df_final1(url, location, stations[0])
    Rail_Station = df_final1(url, location, stations[1])
    Bus_Station = df_final1(url, location, stations[1])
    all_stations_bad_index = pd.concat([Metro, Rail_Station, Bus_Station])
    stations = all_stations_bad_index.reset_index(drop=True)
    return stations

# PARKS

In [22]:
def get_parks(location):
    Parks = df_final1(url, london_loc, "Park")
    return Parks

# DATAFRAME GLOBAL Y LIMPIO

In [23]:
def df_FINAL(location):
    a = get_schools(location, Schools)
    b = get_design_studio(location)
    c = get_tech_startups(location)
    d = get_leisure(location, leisure)
    e = get_closeby_stations(location,stations)
    f = get_parks(location)
    global_dataframe_bad_index = pd.concat([a,b,c,d,e,f])
    global_df_final = global_dataframe_bad_index.reset_index(drop=True)
    return global_df_final

In [24]:
a = df_FINAL(london_loc)

In [40]:
a.sample(10)

,Name,Type,Country,Adress,Latitude,Longitude,location
72,By Creative Ltd,Design Studio,GB,"147, Curtain Rd",51.526614,-0.080559,"{'type': 'Point', 'coordinates': (51.5266135, ..."
104,Knewton London,Tech Startup,GB,89 Worship Place,51.522069,-0.081966,"{'type': 'Point', 'coordinates': (51.522069432..."
76,Fuerte International,Design Studio,GB,6-8 Bonhill Street,51.529646,-0.086316,"{'type': 'Point', 'coordinates': (51.529646321..."
231,The Fox,Pub,GB,28 Paul St.,51.523358,-0.08433,"{'type': 'Point', 'coordinates': (51.523357650..."
40,Harris Academy,High School,GB,55 Southwark Park Road,51.492903,-0.070644,"{'type': 'Point', 'coordinates': (51.492903442..."
51,Alexander McQueen,Design Studio,GB,76-78 Clerkenwell Rd.,51.522614,-0.101735,"{'type': 'Point', 'coordinates': (51.522613794..."
0,Hatching Dragons,Nursery School,GB,1 Viscount Street,51.521599,-0.095356,"{'type': 'Point', 'coordinates': (51.521599411..."
31,Tufnel Park School,Middle School,GB,United Kingdom,51.554251,-0.129383,"{'type': 'Point', 'coordinates': (51.554250946..."
200,Well & Bucket,Pub,GB,143 Bethnal Green Rd,51.525294,-0.070989,"{'type': 'Point', 'coordinates': (51.525293555..."
330,Quaker Garden,Park,GB,Banner St,51.523688,-0.090893,"{'type': 'Point', 'coordinates': (51.523688241..."


# LONDON

In [42]:
london_schools = get_schools(london_loc, Schools)
london_designing_companies = get_design_studio(london_loc)
london_tech_startups = get_tech_startups(london_loc)
london_leisure = get_leisure(london_loc, leisure)
london_travelling_stations = get_closeby_stations(london_loc, stations)
london_parks = get_parks(london_loc)
london_df_entero = df_FINAL(london_loc)

# SINGAPORE

In [43]:
singapore_schools = get_schools(singapore_loc, Schools)
singapore_designing_companies = get_design_studio(singapore_loc)
singapore_tech_startups = get_tech_startups(singapore_loc)
singapore_leisure = get_leisure(singapore_loc, leisure)
singapore_travelling_stations = get_closeby_stations(singapore_loc, stations)
singapore_parks = get_parks(singapore_loc)
singapore_df_entero = df_FINAL(singapore_loc)

# SAN FRANCISCO

In [45]:
sanfrancisco_schools = get_schools(san_francisco_loc, Schools)
sanfrancisco_designing_companies = get_design_studio(san_francisco_loc)
sanfrancisco_tech_startups = get_tech_startups(san_francisco_loc)
sanfrancisco_leisure = get_leisure(san_francisco_loc, leisure)
sanfrancisco_travelling_stations = get_closeby_stations(san_francisco_loc, stations)
sanfrancisco_parks = get_parks(san_francisco_loc)
sanfrancisco_df_entero = df_FINAL(san_francisco_loc)